# Initialize

In [ ]:
!pip install p3_data openpyxl

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pylab as plt
import matplotlib.dates as mdates
import matplotlib.cm as cm
import seaborn as sns
import json
from io import StringIO
import importlib
import re

In [ ]:
import p3_data
from p3_data import (glob_file_list , load_json_from_file, merge_dicts, plot_groups, 
    get_varying_column_names, filter_dataframe, take_varying_columns,
    load_json_records_as_dataframe, flatten_multiindex_columns)

# Load and Clean Results

In [ ]:
# Load result files from P3 Test Driver
src_files = []
src_files += ['../data/p3_test_driver/results/*rapids*.json.bz2']
raw_df = load_json_records_as_dataframe(src=src_files, ignore_error=True)

In [ ]:
r = raw_df[raw_df.test_uuid=='7f98c2f0-7e85-4a55-957c-ba85a1c94cde'].iloc[0]
len(np.array(r.input_files).flatten())
#['input_files'].values.flatten())

In [ ]:
r.input_file_sizes

In [ ]:
# Clean raw results
def clean_result(result):
    try:
        r = result.copy()
        r['utc_begin'] = pd.to_datetime(r['utc_begin'], utc=True)
        r['utc_end'] = pd.to_datetime(r['utc_end'], utc=True)
        if r['single_batch'] == False:
            r['compute_sec'] = np.sum(r['compute_sec_list'])
            r['input_files_size'] = np.sum(r['input_file_sizes'])
            r['num_input_files'] = len(np.array(r.input_files).flatten())
            r['compute_MB_per_sec'] = 1e-6 * r['input_files_size'] / r['compute_sec']
            r['sec_per_batch'] = r['compute_sec'] / r['num_batches']
            r['init_sec'] = r['glob_sec'] + r['getsize_sec'] + r['create_ddf_sec']
        return pd.Series(r)
    except Exception as e:
        print('ERROR: %s: %s' % (r['test_uuid'], e))
        # raise e

In [ ]:
clean_df = raw_df.apply(clean_result, axis=1)
clean_df = clean_df.set_index('test_uuid', drop=False)
clean_df = clean_df[clean_df.error==False]
clean_df['compression'] = clean_df['compression'].fillna('None')
clean_df['warmup'] = clean_df['warmup'].fillna(False)
clean_df = clean_df.sort_values(['utc_begin'])

# Explore data

In [ ]:
# Show list of columns
list(clean_df.columns.values)

In [ ]:
# Define columns that identify test parameters
param_cols = [
 'cached',
 'compression',
 'dask_cudf_version',
 'file_format',
 'input_files_size',
 'num_input_files',
 'num_batches',
 'partitions',
 'size_multiplier',
 'stripe_size_MiB',
 'storage_type',
 'test',
 'warmup',
 'rmm_pool_allocator',
]

In [ ]:
# Define columns that are the output of the experiments
output_cols = [
    'compute_MB_per_sec',
    'compute_sec',
    'init_sec',
    'sec_per_batch',
    'utc_begin',    
]

In [ ]:
cols = param_cols + output_cols

In [ ]:
# View most recent results
clean_df[cols].tail(3).T

In [ ]:
8.40116e+11/2400 * 1e-6

In [ ]:
101/48

In [ ]:
clean_df.head(1)['input_files']

In [ ]:
# Export to CSV
clean_df[cols].to_csv('results.csv')

In [ ]:
filt1_df = filter_dataframe(
    clean_df,
    cached=False,
    compression='snappy',
    data_file_prefix = 'perf-no-strings',
    file_format='orc',
    size_multiplier=1.0,
    stripe_size_MiB=2048,
    utc_begin=(pd.Timestamp('2019-12-13 00:00:00', tz='UTC'),
               pd.Timestamp('2019-12-15 00:00:00', tz='UTC')),
    warmup=False,
    num_batches=50,
    num_input_files=50*48,
    num_workers=48,
)
len(filt1_df)

In [ ]:
take_varying_columns(filt1_df).T

In [ ]:
df = filt1_df.fillna(0).groupby(param_cols).agg({
    'compute_MB_per_sec': ['size','min','mean','max'],
    'compute_sec': ['min','mean','max'],
    'sec_per_batch': ['min','mean','max'],
    'init_sec': ['min','mean','max'],
})
df.T

In [ ]:
18862.231254/5433.366754

In [ ]:
df.unstack(['storage_type'])